<a href="https://colab.research.google.com/github/aylinkarl/Python_MLPractices/blob/main/MNIST_data_set_ML_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
# Let's load the data set.
# 'with_info=True' to get information about version, features and number of instances.
# 'as_supervised=True' to load the dataset in a 2-tuple structure (input, target) 
# 'as_supervised=False' would return a dictionary

mnist_dataset, mnist_info = tfds.load(name="mnist", with_info= True, as_supervised= True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [ ]:
# Training and testing datasets can be easily extracted.

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# I will use 10% of our training dataset as validation.
# 'tf.cast' casts a tensor to a new type.

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1 * mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


# Since the possible values for the inputs are between 0 and 255, 
# we can scale them between 0 and 1 by dividing each element by 255.

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
# 'map' allows us to apply a custom transformation to a given dataset

scaled_train_validation_data = mnist_train.map(scale)

# Let's scale the test data to have the same size as train and validation.
# No need to shuffle as we won't be training with it
# A single batch equal to the size of the test data will suffice.

test_data = mnist_test.map(scale)

In [ ]:
# I will define a buffer size to save memory, this is mostly useful while working big data.
# with this TF only stores BUFFER_SIZE samples in memory at a time and shuffles them.
# If BUFFER_SIZE=1, there will actually be no shuffling, and of course if BUFFER_SIZE >= number of samples, mixing would be uniform.
buf_size = 10000

# I will easily use the shuffle method.
shuffled_train_validation_data = scaled_train_validation_data.shuffle(buf_size)

# I create a batch with a batch size equal to the total number of validation samples
# And the rest would be the train data.

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

# Let's determine a batch size.
batch_size = 50

# batching the train data would be very helpful when we train, as we would be able to iterate over the different batches.
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# 'as_supervized=True' takes the next batch as we have a 2-tuple structure.
validation_inputs, validation_targets = next(iter(validation_data))

In [ ]:
input_size = 784
output_size = 10
# I will use same hidden layer size for all hidden layers
hidden_layer_size = 150

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation="tanh"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="tanh"),
                             tf.keras.layers.Dense(output_size, activation="softmax")
                             ])

In [ ]:
# I will define the optimizer, the loss function, and the metrics to get at each iteration.

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# The maximum number of epochs
max_epochs = 10

# Let's set up an early stop mechanism with patience=2 so our model will be somewhat tolerant of random validation loss increase
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# The accuracy shows in what % of the cases the outputs were equal to the targets.
# The val_accuracy shows the true accuracy of the model.
model.fit(train_data, epochs= max_epochs, callbacks=[early_stopping], validation_data=(validation_inputs,validation_targets), verbose=2 )



Epoch 1/10
1080/1080 - 12s - loss: 0.2412 - accuracy: 0.9254 - val_loss: 0.1304 - val_accuracy: 0.9628 - 12s/epoch - 11ms/step
Epoch 2/10
1080/1080 - 6s - loss: 0.1096 - accuracy: 0.9659 - val_loss: 0.0866 - val_accuracy: 0.9758 - 6s/epoch - 5ms/step
Epoch 3/10
1080/1080 - 6s - loss: 0.0816 - accuracy: 0.9741 - val_loss: 0.0796 - val_accuracy: 0.9762 - 6s/epoch - 5ms/step
Epoch 4/10
1080/1080 - 6s - loss: 0.0647 - accuracy: 0.9789 - val_loss: 0.0690 - val_accuracy: 0.9788 - 6s/epoch - 6ms/step
Epoch 5/10
1080/1080 - 6s - loss: 0.0514 - accuracy: 0.9833 - val_loss: 0.0597 - val_accuracy: 0.9820 - 6s/epoch - 6ms/step
Epoch 6/10
1080/1080 - 6s - loss: 0.0450 - accuracy: 0.9850 - val_loss: 0.0746 - val_accuracy: 0.9770 - 6s/epoch - 5ms/step
Epoch 7/10
1080/1080 - 6s - loss: 0.0398 - accuracy: 0.9872 - val_loss: 0.0685 - val_accuracy: 0.9780 - 6s/epoch - 6ms/step


In [ ]:
# After training on the training data and validating on the validation data, 
# I will test the final prediction power of the model by running it on the test dataset that the algorithm has never seen before.
# The test should be the absolute last sample.

test_loss, test_accuracy = model.evaluate(test_data)
print("test_loss:{0:.2f}, test_accuracy:{1:.2f}%".format(test_loss, test_accuracy*100.))

10/10 [==============================] - 2s 178ms/step - loss: 0.1071 - accuracy: 0.9705
test_loss:0.11, test_accuracy:97.05%


I hope that you find it useful. As I will be happy to improve myself, your comments and feedbacks are always welcome, as are suggestions for additional information that could usefully be included. Thank you! 🌸😊